# Samplers
The samplers module provides various sampling algorithms to generate samples from a given probability distribution. These algorithms can be used to sample a explicit (graph) or implicit ($f(z)=0$) distribution. The samplers can be used for various tasks such as:
* build a dataset or mesh for a manifold
* visualize a distribution
* build a dataset of geodesics using one of the methods in `geodesics.generate`

## Sample from an explicit manifold
The following code samples from an explicit manifold defined by a potential function $\phi(x):\mathbb{R}^i\rightarrow \mathbb{R}^m$, where $i$ is the number of (independent) inputs and $m$ the number of constraints. The dimensionality of ambient space is $n = i+m$.
* Using `randinput_expl` we can obtain a random sample of the manifold by sampling uniformly via latin-hypercube in the input space and projecting the samples onto the manifold using $\phi$. This method can be used to obtain a **uniform sample of the input space**, but it does not guarantee a uniform sample of the manifold itself, but it's fast.
* Using `volume_expl` we can obtain a non-repeated **uniform sample of the manifold**. The function perform an over-sampling of $phi$, which gets then re-sampled $w_i \sim \sqrt{\text{det} [G(x_i)]}$ where $G = J^T J$ is the metric tensor induced by $\phi$.

In [1]:
from jnlr.geodesics.samplers import volume_expl, randinput_expl, langevin_implicit
from jnlr.utils.manifolds import f_ackley as phi
from jnlr.utils.plot_utils import plot_3d_projection

Y_vol, _ = volume_expl(phi, N=5000, oversample=5, roi_R=5)
fig = plot_3d_projection(Y_vol, phi)

# set view from the top
fig.update_layout(scene_camera=dict(eye=dict(x=0., y=0., z=2.5)))


In [2]:

Y_vol = randinput_expl(phi, N=5000, oversample=5, roi_R=5)
fig = plot_3d_projection(Y_vol, phi)
# set view from the top
fig.update_layout(scene_camera=dict(eye=dict(x=0., y=0., z=2.5)))

In [3]:
from jnlr.geodesics.samplers import langevin_implicit

def f(xyz):
    x, y, z = xyz
    return x**4 - x**2 + y**2 + z**2 - 2*x*z
Y_lang = langevin_implicit(f, n=5000, burn=100, thin=2, sigma=0.1, lam=0.01, kappa=0.02, R=3.0)
fig = plot_3d_projection(Y_lang)
fig.update_layout(scene_camera=dict(eye=dict(x=0., y=0., z=2.5)))


In [4]:
import jax.numpy as jnp

def sdf_box(p: jnp.ndarray, b: jnp.ndarray) -> float:
    """Signed distance to an axis–aligned box with half‑sizes ``b``.

    ``p`` is the query point in 3D space.  ``b`` should be a vector of
    positive half–extents along ``x``, ``y`` and ``z``.  Based on
    ``sdBox`` from ``Cube.glsl``.  For a cube of side length 1 the
    appropriate ``b`` is ``(0.5, 0.5, 0.5)``.
    """
    d = jnp.abs(p) - b
    inside = jnp.minimum(jnp.maximum(d[0], jnp.maximum(d[1], d[2])), 0.0)
    outside = jnp.linalg.norm(jnp.maximum(d, 0.0))
    return jnp.array(inside + outside)

def sdf_cube(p: jnp.ndarray, size: float = 1.0) -> float:
    """Convenience wrapper around :func:`sdf_box` for cubes.

    ``size`` is the full length of each side.  The GLSL version uses a
    cube of size 1, corresponding to ``b = (0.5, 0.5, 0.5)``.
    """
    half = 0.5 * size
    b = jnp.array([half, half, half])
    return sdf_box(p, b)


Y_lang = langevin_implicit(sdf_cube, n=1000, burn=100, thin=2, sigma=0.1, lam=0.1, kappa=0.01, R=3.0, tol=1)
fig = plot_3d_projection(Y_lang)
fig.update_layout(scene_camera=dict(eye=dict(x=0., y=0., z=2.5)))

In [5]:
def _max_abs_dot(p: jnp.ndarray, vectors: list[jnp.ndarray]) -> float:
    """Helper to compute the maximum absolute dot product.

    Given a point ``p`` and a list of normal vectors, compute the
    maximum of ``abs(dot(p, v_norm))`` across all normals.  Each
    normal vector in ``vectors`` is normalised inside this function.
    """
    d = 0.0
    for v in vectors:
        v_norm = v / jnp.linalg.norm(v)
        d = jnp.maximum(d, jnp.abs(jnp.dot(p, v_norm)))
    return d

def sdf_dodecahedron(p: jnp.ndarray, scale: float = 0.7, r: float = 1.0) -> float:
    """Signed distance to a dodecahedron.

    The implementation follows ``Dodecahedron.glsl``.  The dodecahedron
    is realised by taking the maximum absolute dot product of ``p`` with
    a set of direction vectors and subtracting the radius ``r``.  A
    uniform ``scale`` factor rescales the space prior to evaluation and
    then re‑applies the inverse to the distance.  The default
    ``scale`` and ``r`` match the GLSL ``sdf`` wrapper.
    """
    # Direction vectors (GDFVector13–18) used in the GLSL code.
    phi = 1.618033988749895
    dirs = [
        jnp.array([0.0,  phi,  1.0]),
        jnp.array([0.0, -phi,  1.0]),
        jnp.array([1.0, 0.0,  phi]),
        jnp.array([-1.0, 0.0,  phi]),
        jnp.array([ phi,  1.0, 0.0]),
        jnp.array([-phi,  1.0, 0.0]),
    ]
    q = p / scale
    d = _max_abs_dot(q, dirs) - r
    return d * scale
Y_lang = langevin_implicit(sdf_dodecahedron, n=5000, burn=100, thin=2, sigma=0.1, lam=0.01, kappa=0.02, R=3.0)
fig = plot_3d_projection(Y_lang)
fig.update_layout(scene_camera=dict(eye=dict(x=0., y=0., z=2.5)))

In [7]:
from jnlr.geodesics.samplers import volume_expl, randinput_expl, langevin_implicit
from jnlr.utils.manifolds import f_ackley as phi
from jnlr.utils.plot_utils import plot_3d_projection
from jnlr.utils.complex_hypersurfaces import surface7

Y_lang = langevin_implicit(surface7, n=10000, burn=100, thin=1, sigma=0.3, lam=0, kappa=0, R=5.0, tol=1e-2)
fig = plot_3d_projection(Y_lang)
fig.update_layout(scene_camera=dict(eye=dict(x=0., y=0., z=2.5)))